<a href="https://colab.research.google.com/github/TatsukiIshijima/Colab_Notebooks/blob/main/TensorFlowTTS_ParallelWaveGAN_with_TFLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TensorFlowTTS の ParallelWaveGAN を TFLite 形式のモデルに変換する

#### TensorFlowTTS をソースからインストールするためにリポジトリをクローン

In [1]:
!git clone https://github.com/TensorSpeech/TensorFlowTTS.git

Cloning into 'TensorFlowTTS'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 10230 (delta 5), reused 8 (delta 1), pack-reused 10207
Receiving objects: 100% (10230/10230), 110.33 MiB | 31.32 MiB/s, done.
Resolving deltas: 100% (4902/4902), done.


ディレクトリの移動

In [2]:
%cd TensorFlowTTS

/content/TensorFlowTTS


#### TensorFlowTTS をインストール

In [3]:
pip install .

Processing /content/TensorFlowTTS
     |████████████████████████████████| 320.4MB 54kB/s 
     |████████████████████████████████| 706kB 54.3MB/s 
     |████████████████████████████████| 245kB 60.6MB/s 
     |████████████████████████████████| 225kB 60.6MB/s 
     |████████████████████████████████| 3.5MB 18.7MB/s 
     |████████████████████████████████| 1.3MB 33.8MB/s 
     |████████████████████████████████| 1.7MB 51.5MB/s 
     |████████████████████████████████| 3.1MB 54.3MB/s 
  Cloning https://github.com/repodiac/german_transliterate.git to /tmp/pip-install-xaxwv005/german-transliterate
  Running command git clone -q https://github.com/repodiac/german_transliterate.git /tmp/pip-install-xaxwv005/german-transliterate
     |████████████████████████████████| 460kB 51.8MB/s 
     |████████████████████████████████| 20.1MB 1.3MB/s 
     |████████████████████████████████| 20.2MB 1.3MB/s 
     |████████████████████████████████| 184kB 62.5MB/s 
     |████████████████████████████████| 102kB 15.5

テキストからメルスペクトグラム変換で tacotron2 を使用するため依存するライブラリをインストールしておく

In [4]:
!pip install tf-nightly

     |████████████████████████████████| 408.1MB 34kB/s 
     |████████████████████████████████| 14.8MB 222kB/s 
     |████████████████████████████████| 5.9MB 45.7MB/s 
     |████████████████████████████████| 471kB 54.5MB/s 
     |████████████████████████████████| 4.0MB 36.4MB/s 
     |████████████████████████████████| 4.0MB 46.6MB/s 
     |████████████████████████████████| 3.8MB 51.2MB/s 
ERROR: umap-learn 0.5.0 has requirement numba>=0.49, but you'll have numba 0.48.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.34.1 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement h5py~=2.10.0, but you'll have h5py 3.1.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorflow-estimator<2.5.0,>=2.4.0, but you'll have tensorflow-estimator 2.3.0 which is incompatible.
ERROR: tensorflow-gpu 2.3.1 has requirement g

#### ParallelWaveGAN をインストール

In [5]:
!pip install parallel_wavegan

     |████████████████████████████████| 51kB 4.2MB/s 
     |████████████████████████████████| 317kB 13.8MB/s 
  Created wheel for parallel-wavegan: filename=parallel_wavegan-0.4.8-cp36-none-any.whl size=54173 sha256=f9c2da1966115160c98df21e543d29dba4f2eb3255a90c52f71366584aa161c0
  Stored in directory: /root/.cache/pip/wheels/47/06/6d/0a45796614b9b7eef98a6eb3a98b201e5715854eaf3b0181bc
  Created wheel for kaldiio: filename=kaldiio-2.17.2-cp36-none-any.whl size=24470 sha256=ca29d895eb632d5ba1a312586a4886bd6681edbfff9944d6ab9b5d8da84aa3d7
  Stored in directory: /root/.cache/pip/wheels/66/9c/1c/6c9ebc779b47f576ffdfdb2bbf8958a67a14ffa41dbef808ca
Successfully built parallel-wavegan kaldiio


#### 必要なライブラリのインポート

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import tensorflow as tf
import torch

from tensorflow_tts.processor import LJSpeechProcessor
from tensorflow_tts.processor.ljspeech import LJSPEECH_SYMBOLS

from tensorflow_tts.configs import ParallelWaveGANGeneratorConfig
from tensorflow_tts.configs import Tacotron2Config

from tensorflow_tts.models import TFTacotron2
from tensorflow_tts.models import TFParallelWaveGANGenerator

from parallel_wavegan.models import ParallelWaveGANGenerator

from IPython.display import Audio
print(tf.__version__)

/usr/local/lib/python3.6/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:43: UserWarning: You are currently using a nightly version of TensorFlow (2.5.0-dev20210214). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  UserWarning,


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
2.5.0-dev20210214


#### 学習モデルの初期化

In [7]:
tf_model = TFParallelWaveGANGenerator(config=ParallelWaveGANGeneratorConfig(), name="parallel_wavegan_generator")

In [8]:
tf_model._build()

#### PyTorch のチェックポイントの読み込み

以下から ljspeech_parallel_wavegan.v1 のチェックポイント等をダウンロードし、Google Drive に保存しておく

https://drive.google.com/drive/folders/1wdHr1a51TLeo4iKrGErVKHVFyq6D17TU

Google Drive から上記のチェックポイント等を読み込みため、Google Drive をマウントする

In [9]:
from google.colab import drive
# Google Drive のマウント
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
torch_checkpoints = torch.load("/content/drive/My Drive/Colab Notebooks/TensorFlowTTS/ljspeech_parallel_wavegan.v1/checkpoint-400000steps.pkl", map_location=torch.device('cpu'))
torch_generator_weights = torch_checkpoints["model"]["generator"]
torch_model = ParallelWaveGANGenerator()
torch_model.load_state_dict(torch_checkpoints["model"]["generator"])
torch_model.remove_weight_norm()

In [11]:
model_parameters = filter(lambda p: p.requires_grad, torch_model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

1334309

#### PyTorch の重みを TensorFlow の重みへ変換

In [12]:
# PyTorch の畳み込み層では順番が バイアス -> 重み、TensorFlow では 重み -> バイアス なので入れ替えが必要

def convert_weights_pytorch_to_tensorflow(weights_pytorch):
    """
    PyTorch の Conv1d（畳み込み）の重み変数を TensorFlow の Conv2D（畳み込み）の重みに変換
    1D: PyTorch (出力、入力、カーネルサイズ) -> TensorFlow（カーネルサイズ、入力、出力）
    2D: PyTorch（出力、入力、カーネルサイズ高さ、カーネルサイズ幅） -> TensorFlow（カーネルサイズ幅、カーネルサイズ高さ、入力、出力）
    """
    if len(weights_pytorch.shape) == 3:  # conv1d-kernel
        weights_tensorflow = np.transpose(weights_pytorch, (0, 2, 1))  #[出力、カーネルサイズ、入力]
        weights_tensorflow = np.transpose(weights_tensorflow, (1, 0, 2))  #[カーネルサイズ、出力、入力]
        weights_tensorflow = np.transpose(weights_tensorflow, (0, 2, 1)) #[カーネルサイズ、入力、出力]
        return weights_tensorflow
    elif len(weights_pytorch.shape) == 1:  # conv1d-bias
        return weights_pytorch
    elif len(weights_pytorch.shape) == 4 : # conv2d-kernel
        weights_tensorflow = np.transpose(weights_pytorch, (0, 2, 1, 3))  #[出力、カーネルサイズ高さ、入力、カーネルサイズ幅]
        weights_tensorflow = np.transpose(weights_tensorflow, (1, 0, 2, 3))  #[カーネルサイズ高さ、出力、入力、カーネルサイズ幅]
        weights_tensorflow = np.transpose(weights_tensorflow, (0, 2, 1, 3))  #[カーネルサイズ高さ、入力、出力、カーネルサイズ幅]
        weights_tensorflow = np.transpose(weights_tensorflow, (0, 1, 3, 2))  #[カーネルサイズ高さ、入力、カーネルサイズ幅、出力]
        weights_tensorflow = np.transpose(weights_tensorflow, (0, 2, 1, 3))  #[カーネルサイズ高さ、カーネルサイズ幅、入力、出力]
        weights_tensorflow = np.transpose(weights_tensorflow, (1, 0, 2, 3))  #[カーネルサイズ幅、カーネルサイズ高さ、入力、出力]
        return weights_tensorflow
    
torch_weights = []
all_keys = list(torch_model.state_dict().keys())
all_values = list(torch_model.state_dict().values())

idx_already_append = []

for i in range(len(all_keys) - 1):
    if i not in idx_already_append:
        if all_keys[i].split(".")[0:-1] == all_keys[i+1].split(".")[0:-1]:
            if all_keys[i].split(".")[-1] == "bias" and all_keys[i+1].split(".")[-1] == "weight":
                torch_weights.append(convert_weights_pytorch_to_tensorflow(all_values[i+1].cpu().detach().numpy()))
                torch_weights.append(convert_weights_pytorch_to_tensorflow(all_values[i].cpu().detach().numpy()))
                idx_already_append.append(i)
                idx_already_append.append(i+1)
        else:
            if i not in idx_already_append:
                torch_weights.append(convert_weights_pytorch_to_tensorflow(all_values[i].cpu().detach().numpy()))
                idx_already_append.append(i)

In [13]:
tf_var = tf_model.trainable_variables
for i, var in enumerate(tf_var):
    tf.keras.backend.set_value(var, torch_weights[i])

#### TFLite 形式へ変換

In [14]:
def convert_to_tflite(quantization):
    pwg_concreate_function = tf_model.inference.get_concrete_function()
    converter = tf.lite.TFLiteConverter.from_concrete_functions([pwg_concreate_function])
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_ops = [tf.lite.OpsSet.SELECT_TF_OPS]
    if quantization == 'float16':
        converter.target_spec.supported_types = [tf.float16]
    tf_lite_model = converter.convert()
    model_name = f'parallel_wavegan_{quantization}.tflite'
    with open(f'/content/drive/My Drive/Colab Notebooks/TensorFlowTTS/ljspeech_parallel_wavegan.v1/{model_name}', 'wb') as f:
        f.write(tf_lite_model)

##### ダイナミックレンジ量子化

In [15]:
quantization = 'dr' #@param ["dr", "float16"]
convert_to_tflite(quantization)

In [16]:
!du -sh "/content/drive/My Drive/Colab Notebooks/TensorFlowTTS/ljspeech_parallel_wavegan.v1/parallel_wavegan_dr.tflite"

5.7M	/content/drive/My Drive/Colab Notebooks/TensorFlowTTS/ljspeech_parallel_wavegan.v1/parallel_wavegan_dr.tflite


##### 半精度浮動小数点数量子化

In [17]:
quantization = 'float16'
convert_to_tflite(quantization)

In [18]:
!du -sh "/content/drive/My Drive/Colab Notebooks/TensorFlowTTS/ljspeech_parallel_wavegan.v1/parallel_wavegan_float16.tflite"

3.1M	/content/drive/My Drive/Colab Notebooks/TensorFlowTTS/ljspeech_parallel_wavegan.v1/parallel_wavegan_float16.tflite


## TFLite 形式のParallelWaveGAN を使ってみる

ParallelWaveGAN はメルスペクトグラムから音声への変換なので、その前段階のテキストからメルスペクトグラム変換に tacotron2 を使用する

In [19]:
# Tacotron2 の TFLite model の読み込み
tacotron2_interpreter = tf.lite.Interpreter(model_path='/content/drive/My Drive/Colab Notebooks/TensorFlowTTS/tacotron2.v1/tacotron2.tflite')
tacotron2_interpreter.allocate_tensors()

# 入力と出力のテンソルを取得
input_details = tacotron2_interpreter.get_input_details()
output_details = tacotron2_interpreter.get_output_details()

print('input_details : ', input_details)
print('output_details : ', output_details)

# 入力データの準備
def prepare_input(input_ids):
    return (tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
            tf.convert_to_tensor([len(input_ids)], tf.int32),
            tf.convert_to_tensor([0], dtype=tf.int32))
    
# 適当な入力データによるモデルの動作確認
def infer(input_text):
    processor = LJSpeechProcessor(None, symbols=LJSPEECH_SYMBOLS)
    input_ids = processor.text_to_sequence(input_text.lower())
    tacotron2_interpreter.resize_tensor_input(input_details[0]['index'], [1, len(input_ids)])
    tacotron2_interpreter.allocate_tensors()
    input_data = prepare_input(input_ids)
    for i, detail in enumerate(input_details):
        print(detail)
        input_shape = detail['shape']
        tacotron2_interpreter.set_tensor(detail['index'], input_data[i])
    
    tacotron2_interpreter.invoke()

    return (tacotron2_interpreter.get_tensor(output_details[0]['index']),
            tacotron2_interpreter.get_tensor(output_details[1]['index']))

input_details :  [{'name': 'input_ids', 'index': 0, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([ 1, -1], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'input_lengths', 'index': 1, 'shape': array([1], dtype=int32), 'shape_signature': array([1], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'speaker_ids', 'index': 2, 'shape': array([1], dtype=int32), 'shape_signature': array([1], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_pa

#### Tacotron2 の出力のサンプルデータのダウンロード

Tacotron2 の出力のサンプルデータを ParallelWaveGAN に入力して試してみる

In [22]:
!gdown --id 1LmU3j8yedwBzXKVDo9tCvozLM4iwkRnP -O tac_output.npy

Downloading...
From: https://drive.google.com/uc?id=1LmU3j8yedwBzXKVDo9tCvozLM4iwkRnP
To: /content/TensorFlowTTS/tac_output.npy
100% 36.0k/36.0k [00:00<00:00, 16.8MB/s]


In [25]:
parallel_wavegan_interpreter = tf.lite.Interpreter(model_path='/content/drive/My Drive/Colab Notebooks/TensorFlowTTS/ljspeech_parallel_wavegan.v1/parallel_wavegan_dr.tflite')

parallel_input_details = parallel_wavegan_interpreter.get_input_details()
parallel_output_details = parallel_wavegan_interpreter.get_output_details()

print('input_details : ', parallel_input_details)
print('output_details : ', parallel_output_details)

data = np.load('tac_output.npy')
feats = np.expand_dims(data, 0)

parallel_wavegan_interpreter.resize_tensor_input(parallel_input_details[0]['index'], [1, feats.shape[1], feats.shape[2]], strict=True)
parallel_wavegan_interpreter.allocate_tensors()

parallel_wavegan_interpreter.set_tensor(parallel_input_details[0]['index'], feats)

parallel_wavegan_interpreter.invoke()

parallel_output = parallel_wavegan_interpreter.get_tensor(parallel_output_details[0]['index'])

input_details :  [{'name': 'mels', 'index': 0, 'shape': array([ 1,  1, 80], dtype=int32), 'shape_signature': array([-1, -1, 80], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
output_details :  [{'name': 'Identity', 'index': 1958, 'shape': array([1, 1, 1], dtype=int32), 'shape_signature': array([-1, -1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
0
112
80
(1, 112, 80)


In [26]:
parallel_output = parallel_output[0, :, 0]
Audio(parallel_output, rate=22050)

#### TFLite 形式の Tacotron2 からの出力を ParallelWaveGAN に入力して試す

In [27]:
input_text = "Recent research at Harvard has shown meditating for as little as 8 weeks, can actually increase the grey matter in the parts of the brain responsible for emotional regulation, and learning."

decoder_output_tffile, mel_output_tflite = infer(input_text)
print(decoder_output_tffile.shape)

{'name': 'input_ids', 'index': 0, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([ 1, -1], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
{'name': 'input_lengths', 'index': 1, 'shape': array([1], dtype=int32), 'shape_signature': array([1], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
{'name': 'speaker_ids', 'index': 2, 'shape': array([1], dtype=int32), 'shape_signature': array([1], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
(1, 9

In [28]:
parallel_wavegan_interpreter.resize_tensor_input(parallel_input_details[0]['index'], [1, decoder_output_tffile.shape[1], decoder_output_tffile.shape[2]], strict=True)
parallel_wavegan_interpreter.allocate_tensors()

parallel_wavegan_interpreter.set_tensor(parallel_input_details[0]['index'], decoder_output_tffile)

parallel_wavegan_interpreter.invoke()

parallel_output = parallel_wavegan_interpreter.get_tensor(parallel_output_details[0]['index'])

In [29]:
parallel_output = parallel_output[0, :, 0]
Audio(parallel_output, rate=22050)